In [39]:
# !pip install selenium
# !pip install underthesea # work_tokenize, sentiment
# ! pip install transformers

In [76]:
import urllib.request
from bs4 import BeautifulSoup # parse html
import re #regex
import csv
import os
import json
import pandas as pd
import urllib.request
import joblib
from underthesea import word_tokenize #word_tokenize of lines
import numpy as np
import transformers as ppb # load model BERT
from transformers import BertModel, BertTokenizer
import torch
from selenium.webdriver.common.keys import Keys
from sklearn.model_selection import train_test_split
# scrap comment = selenium
from selenium import webdriver 
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException, WebDriverException
import time
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
# import requests

# Craw comment of product tiki, lazada

In [77]:


def load_url_selenium_lazada(url):
    # Selenium
    driver = webdriver.Chrome(
        executable_path="C:/Users/DE/Desktop/Crawler/chromedriver.exe")
    print("Loading url=", url)
    driver.get(url)
    list_review = []
    # just craw 10 page
    # x=0
    # while x<10:
    #     # try:
    #     #     #Get the review details here
    #     #     WebDriverWait(driver,5).until(EC.visibility_of_all_elements_located((By.CSS_SELECTOR,".mod-reviews")))
    #     # except:
    #     #     print('No comment')
    #     #     break
    #     # x +=1
    #     product_reviews = driver.find_elements_by_css_selector("[class='item']")
    #     # Get product review
    #     for product in product_reviews:
    #         review = product.find_element_by_css_selector("[class='content']").text
    #         if (review != "" or review.strip()):
    #             print(review, "\n")
    #             list_review.append(review)
    #     #Check for button next-pagination-item have disable attribute then jump from loop else click on the next button
    #     if len(driver.find_elements_by_css_selector("button.next-pagination-item.next[disabled]"))>0:
    #         break
    #     else:
    #         button_next=WebDriverWait(driver, 5).until(EC.visibility_of_element_located((By.CSS_SELECTOR, "button.next-pagination-item.next")))
    #         driver.execute_script("arguments[0].click();", button_next)
    #         print("next page")
    #         time.sleep(2)
    #         x +=1
    driver.close()
    return list_review


def load_url_selenium_tiki(url):
    driver = webdriver.Chrome(
        executable_path="C:/Users/DE/Desktop/Crawler/chromedriver.exe")
    print("Loading url=", url)
    driver.get(url)
    list_review = []
    # just craw 10 page
    x = 0
    while x < 10:
        last = None
        for v in range(500):
            for k in range(5):
                driver.find_element_by_xpath('//html').send_keys(Keys.DOWN)

            if last is not None and last == driver.execute_script('return window.pageYOffset;'):
                break
            last = driver.execute_script('return window.pageYOffset;')

        comment = driver.find_elements(By.XPATH, value="//div[@class='review-comment__content']")

        for review in comment:
            review_comment = review.text
            if (review_comment != "" or review_comment.strip()):
                list_review.append(review_comment)
        
        
        
        # Check for button next-pagination-item have disable attribute then jump from loop else click on the next button
        try:
            driver.find_element_by_xpath("//a[@class='btn next']").click()
            print("next page")
            time.sleep(2)
            x += 1
        except (TimeoutException, WebDriverException) as e:
            print('Load several page!')
            break
    driver.close()
    return list_review


# Standard data, tokenizer

In [78]:

def standardize_data(row):
    # remove stopword
    # Remove . ? , at index final
    row = re.sub(r"[\.,\?]+$-", "", row)
    # Remove all . , " ... in sentences
    row = row.replace(",", " ").replace(".", " ") \
        .replace(";", " ").replace("“", " ") \
        .replace(":", " ").replace("”", " ") \
        .replace('"', " ").replace("'", " ") \
        .replace("!", " ").replace("?", " ") \
        .replace("-", " ").replace("?", " ")

    row = row.strip()
    return row

# Tokenizer
def tokenizer(row):
    return word_tokenize(row, format="text")

def analyze(result):
    bad = np.count_nonzero(result)
    good = len(result) - bad
    print("No of bad and neutral comments = ", bad)
    print("No of good comments = ", good)

    if good>bad:
        return "Good! You can buy it!"
    else:
        return "Bad! Please check it carefully!"


# Processing data

In [79]:
def processing_data(data):
    # 1. Standardize data
    data_frame = pd.DataFrame(data)
    print('data frame:', data_frame)
    data_frame[0] = data_frame[0].apply(standardize_data)

    # 2. Tokenizer
    data_frame[0] = data_frame[0].apply(tokenizer)

    # 3. Embedding
    X_val = data_frame[0]
    return X_val


# Load Pretrain model BERT

In [80]:
def load_pretrainModel(data):
    
    '''
    Load pretrain model/ tokenizers
    Return : features
    '''
    model = BertModel.from_pretrained('bert-base-uncased')
    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

    #encode lines
    tokenized = data.apply((lambda x: tokenizer.encode(x, add_special_tokens = True)))

    # get lenght max of tokenized
    max_len = 0
    for i in tokenized.values:
        if len(i) > max_len:
            max_len = len(i)
    print('max len:', max_len)

    # if lenght of tokenized not equal max_len , so padding value 0
    padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values])
    print('padded:', padded[1])
    print('len padded:', padded.shape)

    #get attention mask ( 0: not has word, 1: has word)
    attention_mask = np.where(padded ==0, 0,1)
    print('attention mask:', attention_mask[1])

    # Convert input to tensor
    padded = torch.tensor(padded)
    attention_mask = torch.tensor(attention_mask)


    # Load model
    with torch.no_grad():
        last_hidden_states = model(padded, attention_mask =attention_mask)
    #     print('last hidden states:', last_hidden_states)

    features = last_hidden_states[0][:,0,:].numpy()
    print('features:', features)
    
    return features


# Predict

In [81]:
def predict(url):
    # 1. Load URL and print comments
    if url== "":
        url = "https://tiki.vn/dien-thoai-samsung-galaxy-m31-128gb-6gb-hang-chinh-hang-p58259141.html"
    # data = load_url_selenium_lazada(url)
    data = load_url_selenium_tiki(url)
    print(data)
    a_file = open("data_crawler.csv", "w",  encoding='utf-8')
    for row in data:
        a_file.write(row + "\n")

    a_file.close()
    data = processing_data(data)
    features = load_pretrainModel(data)
    # 2. Load weights
    model = joblib.load('save_model.pkl')
    # 3. Result
    result = model.predict(features)
    print(result)
    print(analyze(result))

predict(url ='https://tiki.vn/combo-dau-goi--dau-xa-tresemme-keratin-smooth-duong-toc-kho-xo-roi-cong-thuc-chuan-salon-hydrolyzed-keratin-trong-dau-goi-va-dau-xa-duong-toc-vao-nep-suon-muot-640g--620g-p22254660.html?spid=22254661')
# predict(url = 'https://www.lazada.vn/products/op-lung-iphone-6-6s-plus-7-plus-8-plus-x-xr-xsmax-11-11-pro-11-promax-12-12-pro-12-promax-13-promax-trong-suot-chong-soc-i1277055288-s7560624279.html?search=1')

C:\Users\DE\AppData\Local\Temp\ipykernel_7740\1979255434.py:39: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(


Loading url= https://tiki.vn/combo-dau-goi--dau-xa-tresemme-keratin-smooth-duong-toc-kho-xo-roi-cong-thuc-chuan-salon-hydrolyzed-keratin-trong-dau-goi-va-dau-xa-duong-toc-vao-nep-suon-muot-640g--620g-p22254660.html?spid=22254661


C:\Users\DE\AppData\Local\Temp\ipykernel_7740\1979255434.py:50: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  driver.find_element_by_xpath('//html').send_keys(Keys.DOWN)
C:\Users\DE\AppData\Local\Temp\ipykernel_7740\1979255434.py:67: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  driver.find_element_by_xpath("//a[@class='btn next']").click()


next page
next page
next page
next page
next page
next page
next page
next page
next page
next page
['Mở hộp ra thấy mùi thơm dễ chịu, đóng gói cẩn thận. Chưa dùng nên chưa bik chất lượng sản phẩm ntn. Có quà tặng đi kèm. Nói chung chim ưng okokokokokokokokeuwuwuwuwiieiwuwiwiwiwiwiwiwieieieieieiejejejeieieieoeoeoeoeoeeoeowooeoeeoeooeoeoeoeowowowowowoeoeoeoeoeoeoeoeoeoeoeoeoeoe', 'TIKI giao hàng maxxxxx nhanh luôn ạ. 10h tối đặt, 10h sáng đã gọi xuống nhận hàng. Đóng gói cẩn thận, ko dùng túi chống sốc nilon mà dùng thùng giấy tái sử dụng ???. Săn sale chính hãng giá rẻ, sản phẩm mùi thơm hơi nặng, tóc bóng mượt, ko bị khô.', 'Giao hàng siêu nhanh, tối hôm trước hôm sau đã có hàng. Sản phẩm được tặng kèm nhiều thứ . Dầu gội thơm, nhưng dùng dầu xả thì phải xả kĩ không tóc dễ bết dầu. Dùng 1 thời gian chưa thấy tóc chẻ ngọn. Mới dùng thì sau khi sấy xong tóc khá mượt. Được nửa chai bắt đầu thấy có vẻ hơi khô so với loại trước mình dùng.', 'MÓN QUÀ Ý NGHĨA CỦA KHÓA SINH CLB LA HẦU LA CÚNG

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


max len: 277
padded: [  101 14841  3211 27699  2080  6865  4098 20348 20348 18699  2319  2232
 11320  2239  1037  2184  1044  2000  2072  1102  4017  2184  1044  6369
  1102  2050  2175  2072 15990  5063 18699  2319  6865  1102  5063  1035
  2175  2072  2064  1035  2084 12849 29328 10722  2072 24008 27084  9152
  7811  5003 29328 16215  5575 27699  2100 13843 10514  1035 29328  2624
  5096  5413  2232  6865 27699  2128  2624  1035  6887  3286 14163  2072
 19438  7570  2072 16660  2290  2000  2278 14753  2290 14163  4140 12849
 12170  1047  6806   102     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0